In [33]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.text import one_hot
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [34]:
movie_data = pd.read_csv(r'C:\Users\satya\Downloads\sentiment-analysis-on-movie-reviews\train.csv')
movie_test = pd.read_csv(r'C:\Users\satya\Downloads\sentiment-analysis-on-movie-reviews\test.csv')
movie_sub = pd.read_csv(r'C:\Users\satya\Downloads\sentiment-analysis-on-movie-reviews\sampleSubmission.csv')

In [35]:
movie_data

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [36]:
def preprocess2(phrase, new_col):
    def remove_pattern(input_txt, pattern):
        r = re.findall(pattern, input_txt)
        for word in r:
            input_txt = re.sub(word, "", input_txt)
        return input_txt
    movie_data[new_col] = np.vectorize(remove_pattern)(phrase, "@[\w]*")
    movie_data[new_col] = movie_data[new_col].str.replace("[^a-zA-Z#]", " ")
    movie_data[new_col] = movie_data[new_col].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
    tokenized_tweet = movie_data[new_col].apply(lambda x: x.split())
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()

    tokenized_tweet = tokenized_tweet.apply(lambda sentence: [stemmer.stem(word) for word in sentence])
    for i in range(len(tokenized_tweet)):
        tokenized_tweet[i] = " ".join(tokenized_tweet[i])
    
    movie_data[new_col] = tokenized_tweet
    return movie_data[new_col]

In [9]:
preprocess2(movie_data['Phrase'],'clean_phrase')
movie_data

C:\Users\satya\AppData\Local\Temp\ipykernel_11040\3865188208.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  movie_data[new_col] = movie_data[new_col].str.replace("[^a-zA-Z#]", " ")


,PhraseId,SentenceId,Phrase,Sentiment,clean_phrase
0,1,1,A series of escapades demonstrating the adage ...,1,seri escapad demonstr adag that what good goos...
1,2,1,A series of escapades demonstrating the adage ...,2,seri escapad demonstr adag that what good goos
2,3,1,A series,2,seri
3,4,1,A,2,
4,5,1,series,2,seri
...,...,...,...,...,...
156055,156056,8544,Hearst 's,2,hearst
156056,156057,8544,forced avuncular chortles,1,forc avuncular chortl
156057,156058,8544,avuncular chortles,3,avuncular chortl
156058,156059,8544,avuncular,2,avuncular


In [10]:
movie_data['clean_phrase'].isnull().sum()

0

In [11]:
voc_size=100000
onehot_repr1=[one_hot(phrase,voc_size)for phrase in movie_data['clean_phrase']] 
onehot_repr2=[one_hot(phrase,voc_size)for phrase in movie_data['Phrase']]

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [16]:
length=[]
for x in movie_data['Phrase']:
    length.append(len(x.split(' ')))
max_length=max(length)
max_length

52

In [18]:
sent_length=max_length+10
embedded_docs1=pad_sequences(onehot_repr1,padding='pre',maxlen=sent_length)
embedded_docs2=pad_sequences(onehot_repr2,padding='pre',maxlen=sent_length)
print(embedded_docs1)

[[    0     0     0 ... 75358 96831 49969]
 [    0     0     0 ...  8794 23241 26199]
 [    0     0     0 ...     0     0  3954]
 ...
 [    0     0     0 ...     0 88574 20770]
 [    0     0     0 ...     0     0 88574]
 [    0     0     0 ...     0     0 20770]]


In [19]:
movie_data.Sentiment.value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [42]:
x_final1 = np.array(embedded_docs1)
x_final2 = np.array(embedded_docs2)
y_final = np.array(movie_data['Sentiment'])
x_train1,x_test1,y_train1,y_test1 = train_test_split(x_final1,y_final,test_size=0.33, random_state=42)
x_train2,x_test2,y_train2,y_test2 = train_test_split(x_final2,y_final,test_size=0.33, random_state=42)

In [30]:
sent_length=max_length+10
embedding_vector_features=10 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dense(64, activation='relu'))
model.add(LSTM(100))
model.add(Dense(5,activation='softmax'))
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),optimizer='adam',metrics=['accuracy'])

In [31]:
model.fit(x_train1,y_train1,validation_data=(x_test1,y_test1),epochs=10)

Epoch 1/10
3268/3268 [==============================] - 283s 85ms/step - loss: 1.0215 - accuracy: 0.5894 - val_loss: 1.2117 - val_accuracy: 0.5331
Epoch 2/10
3268/3268 [==============================] - 160s 49ms/step - loss: 0.8713 - accuracy: 0.6454 - val_loss: 1.2641 - val_accuracy: 0.4778
Epoch 3/10
3268/3268 [==============================] - 158s 48ms/step - loss: 0.8273 - accuracy: 0.6611 - val_loss: 1.3589 - val_accuracy: 0.4616
Epoch 4/10
3268/3268 [==============================] - 157s 48ms/step - loss: 0.7976 - accuracy: 0.6721 - val_loss: 1.3249 - val_accuracy: 0.4633
Epoch 5/10
3268/3268 [==============================] - 162s 50ms/step - loss: 0.7735 - accuracy: 0.6808 - val_loss: 1.2835 - val_accuracy: 0.4600
Epoch 6/10
3268/3268 [==============================] - 164s 50ms/step - loss: 0.7521 - accuracy: 0.6883 - val_loss: 1.3154 - val_accuracy: 0.4477
Epoch 7/10
3268/3268 [==============================] - 163s 50ms/step - loss: 0.7291 - accuracy: 0.6977 - val_loss: 1

In [56]:
embedding_vector_features=10 ##features representation
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(128,activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(64,activation='relu'))
model2.add(Dropout(0.1))
model2.add(LSTM(20))
model2.add(Dense(5,activation='softmax'))
model2.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),optimizer='adam',metrics=['accuracy'])

In [59]:
model.fit(x_train1,y_train1,validation_data=(x_test1,y_test1),epochs=10)

Epoch 1/10
3268/3268 [==============================] - 193s 59ms/step - loss: 0.6581 - accuracy: 0.7255 - val_loss: 0.9365 - val_accuracy: 0.6457
Epoch 2/10
3268/3268 [==============================] - 194s 59ms/step - loss: 0.6408 - accuracy: 0.7321 - val_loss: 0.9490 - val_accuracy: 0.6453
Epoch 3/10
3268/3268 [==============================] - 190s 58ms/step - loss: 0.6258 - accuracy: 0.7380 - val_loss: 0.9554 - val_accuracy: 0.6433
Epoch 4/10
3268/3268 [==============================] - 185s 57ms/step - loss: 0.6103 - accuracy: 0.7438 - val_loss: 0.9784 - val_accuracy: 0.6384
Epoch 5/10
3268/3268 [==============================] - 187s 57ms/step - loss: 0.5966 - accuracy: 0.7484 - val_loss: 1.0086 - val_accuracy: 0.6426
Epoch 6/10
3268/3268 [==============================] - 196s 60ms/step - loss: 0.5834 - accuracy: 0.7548 - val_loss: 1.0064 - val_accuracy: 0.6431
Epoch 7/10
3268/3268 [==============================] - 194s 59ms/step - loss: 0.5704 - accuracy: 0.7589 - val_loss: 1

In [62]:
predictions=model.predict(x_test1)

1610/1610 [==============================] - 24s 15ms/step


array([8.2195284e-06, 1.1444017e-01, 8.8500178e-01, 5.4969615e-04,
       1.4035220e-07], dtype=float32)

array([2, 4, 2, ..., 1, 1, 3], dtype=int64)

In [60]:
movie_test

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
...,...,...,...
66287,222348,11855,"A long-winded , predictable scenario ."
66288,222349,11855,"A long-winded , predictable scenario"
66289,222350,11855,"A long-winded ,"
66290,222351,11855,A long-winded


In [69]:
def preprocess(phrase, new_col):
    def remove_pattern(input_txt, pattern):
        r = re.findall(pattern, input_txt)
        for word in r:
            input_txt = re.sub(word, "", input_txt)
        return input_txt
    movie_test[new_col] = np.vectorize(remove_pattern)(phrase, "@[\w]*")
    movie_test[new_col] = movie_test[new_col].str.replace("[^a-zA-Z#]", " ")
    movie_test[new_col] = movie_test[new_col].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
    tokenized_tweet = movie_test[new_col].apply(lambda x: x.split())
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()

    tokenized_tweet = tokenized_tweet.apply(lambda sentence: [stemmer.stem(word) for word in sentence])
    for i in range(len(tokenized_tweet)):
        tokenized_tweet[i] = " ".join(tokenized_tweet[i])
    
    movie_test[new_col] = tokenized_tweet
    return movie_test[new_col]

In [70]:
preprocess(movie_test['Phrase'],'clean_phrase_test')
movie_test

C:\Users\satya\AppData\Local\Temp\ipykernel_11040\1749866064.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  movie_test[new_col] = movie_test[new_col].str.replace("[^a-zA-Z#]", " ")


,PhraseId,SentenceId,Phrase,clean_phrase_test
0,156061,8545,An intermittently pleasing but mostly routine ...,intermitt pleas mostli routin effort
1,156062,8545,An intermittently pleasing but mostly routine ...,intermitt pleas mostli routin effort
2,156063,8545,An,
3,156064,8545,intermittently pleasing but mostly routine effort,intermitt pleas mostli routin effort
4,156065,8545,intermittently pleasing but mostly routine,intermitt pleas mostli routin
...,...,...,...,...
66287,222348,11855,"A long-winded , predictable scenario .",long wind predict scenario
66288,222349,11855,"A long-winded , predictable scenario",long wind predict scenario
66289,222350,11855,"A long-winded ,",long wind
66290,222351,11855,A long-winded,long wind


In [73]:
voc_size=100000
onehot_repr1_test=[one_hot(phrase,voc_size)for phrase in movie_test['clean_phrase_test']] 
onehot_repr2_test=[one_hot(phrase,voc_size)for phrase in movie_test['Phrase']]

In [74]:
sent_length=70
embedded_docs1_test=pad_sequences(onehot_repr1_test,padding='pre',maxlen=sent_length)
embedded_docs2_test2=pad_sequences(onehot_repr2_test,padding='pre',maxlen=sent_length)
print(embedded_docs1)

[[    0     0     0 ... 39489 57784 85493]
 [    0     0     0 ... 39489 57784 85493]
 [    0     0     0 ...     0     0     0]
 ...
 [    0     0     0 ...     0 34047 17286]
 [    0     0     0 ...     0 34047 17286]
 [    0     0     0 ...     0 11085 85339]]


In [75]:
test1 = np.array(embedded_docs1)
test2 = np.array(embedded_docs2)

In [103]:
len(test1)

66292

In [78]:
test_prediction1 = model.predict(test1)

2072/2072 [==============================] - 44s 21ms/step


In [79]:
test_prediction2 = model.predict(test2)

2072/2072 [==============================] - 43s 21ms/step


In [84]:
labels1=[]
for x in test_prediction1:
    label =np.argmax(x)
    labels1.append(label)
sentiments1=np.array(labels1)


In [85]:
labels2=[]
for x in test_prediction2:
    label =np.argmax(x)
    labels2.append(label)
sentiments2=np.array(labels2)

In [96]:
pwd

'C:\\Users\\satya'

In [101]:
movie_sub['Sentiment']=sentiments1
movie_sub.to_csv('sub1.csv',index=False)

In [102]:
movie_sub['Sentiment']=sentiments2
movie_sub.to_csv('sub2.csv',index=False)